In [5]:
import spacy
import json

import os
import openai

from tqdm import tqdm
import time

import numpy as np
np.random.seed(0)

openai.api_key = 

dev = json.load(open("../data/individual_dev_end2end_final.json"))

get_lemma = lambda x:" ".join([token.lemma_ for token in nlp(x)])

nlp = spacy.load("en_core_web_sm")

relation_trigger = ['before', 'after', 'during', 'while']

warmup_qs = ['What will happen in the future?', 'What event has already finished?', 
             'What event has begun but has not finished?', 'What is happening now?',
             'What event has already happened?', 'What event has started?', 'What events have already finished?']

def parse_question(q, event_lemmas):
    """
        input: q: question, events: the set of lemmatized events.
        output: 
            q_events: events in the question
            modality: whether there's "might/will/can/may/..."
            base_temp_rel: basic temporal relations, ["before", "after", "during", "while"]
    """
    # acquire the events in the question stem
    q_events = [e for e in [token.lemma_ for token in nlp(q)] if e in event_lemmas]
    
    second_prefix = q.split()[1]

    rel_trigger = [t for t in q.split() if t in relation_trigger]

    if len(rel_trigger) > 0:
        base_temp_rel = rel_trigger[0]
    else:
        base_temp_rel = ""
        
    return q_events, second_prefix, base_temp_rel

question_generator = {
    'What will happen in the future?': 'will happen in the future', 
    'What event has already finished?': 'have happened', 
    'What events have already finished?': 'have happened', 
    'What event has begun but has not finished?': 'have begun but have not finished', 
    'What events have begun but has not finished?' : 'have begun but have not finished', 
    'What is happening now?': 'is happening now',
    'What event has already happened?': 'have happened', 
    'What event has started?':'have started happening', 
                     }

In [3]:
oneshot_gen_results = json.load(open("results/template_1_oneshot_0_pred"))

In [24]:
np.random.seed(0)
generated_examplars_cf = {}
generated_examplars_answers_cf = {}
examplar_prompts_cf = {}

for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c.lower() for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c.lower() for c, t in zip(item['context'], item["answers"]["labels"]) if t]

    event_lemmas = [get_lemma(e) for e in all_events]
    
    if not question in question_generator:
        question_events, _, rel = parse_question(question, event_lemmas)
        gpt3_answers = [w.lower().strip().replace(".", "") for w in oneshot_gen_results[key].split(",")]
        selected_ans = list(set(all_events) - set(gpt3_answers))
#         print(len(selected_ans), len(gpt3_answers), len(all_events))
#         print(selected_ans, gpt3_answers, all_events)
        if len(question_events) > 0:
            prompt = f"Write a story where '{', '.join(selected_ans)}' happened {rel} '{question_events[0]}' within 100 words:"
            question = f"What happened {rel} {question_events[0]}?"
        else:
            question = item['question']
            prompt = f"Write a story involving where the answer to the question {question} is '{', '.join(selected_ans)}' within 100 words:"

    else:
        
        gpt3_answers = [w.lower().strip().replace(".", "") for w in oneshot_gen_results[key].split(",")]
        selected_ans = list(set(all_events) - set(gpt3_answers))
        prompt = f"Write a story where '{', '.join(selected_ans)}' {question_generator[question]} within 100 words:"
        question = item['question']
    
    generated_examplars_answers_cf[key] = selected_ans
    generated_examplars_cf[key] = ""
    while True:
        try:
            generated_examplars_cf[key] = openai.Completion.create(
                                                  model="text-davinci-003",
                                                  prompt=prompt,
                                                  max_tokens=100,
                                                  temperature=0
                                        )["choices"][0]["text"].strip()
            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)
    
    
    context = generated_examplars_cf[key]
    gen_answers = generated_examplars_answers_cf[key]
    ex_prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA: " + ', '.join(gen_answers)

    examplar_prompts_cf[key] = ex_prompt

  0%|▍                                                                                                                 | 6/1483 [00:42<2:55:12,  7.12s/it]


KeyboardInterrupt: 

In [20]:
question

'What could have happened after the fighting of the LRA?'

In [ ]:
cda_results_0 = {}
for key, item in tqdm(dev.items()):
    context = " ".join(item['context'])
    question = item['question']
    all_events = [c for c, t in zip(item['context'], item["answers"]["types"]) if t]
    ground = [c for c, t in zip(item['context'], item["answers"]["labels"]) if t]
    prompt = f"Q: {question}" + ", select none or several from {" + ', '.join(all_events) + "} \n" + context + "\nA:"
    
    while True:
        try:
            cda_results_0[key] = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=examplar_prompts_cf[key] + "\n\n" + three_shot_exemplar + "\n\n" + prompt,
                      max_tokens=256,
                      temperature=0
            )["choices"][0]["text"].strip()

            time.sleep(1)
            break
        except Exception as e:
            print(e)
            time.sleep(10)